In [1]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime
from typing import List, Dict, Any
from enum import Enum
import json

# MongoDB connection setup
client = MongoClient('mongodb://localhost:27017/')
db = client['mdm_example']

# Enum for Rules
class RuleType(str, Enum):
    conflict_resolution = "conflict_resolution"
    data_aggregation = "data_aggregation"
    survivorship = "survivorship"

def load_data(collection_name):
    collection = db[collection_name]
    data = list(collection.find({}))
    return pd.DataFrame(data)

def normalize_columns(df, id_col, name_col, email_col, **kwargs):
    column_mapping = {
        id_col: 'id',
        name_col: 'name',
        email_col: 'email'
    }
    column_mapping.update(kwargs)
    return df.rename(columns=column_mapping)

In [3]:
def resolve_conflicts(records: List[Dict[str, Any]]) -> Dict[str, Any]:
    golden_record = {}
    for field in records[0].keys():
        if field == 'id':
            golden_record[field] = records[0][field]
        else:
            values = [record[field] for record in records if pd.notna(record[field])]
            if values:
                golden_record[field] = max(set(values), key=values.count)
    return golden_record

def aggregate_data(records: List[Dict[str, Any]]) -> Dict[str, Any]:
    golden_record = {}
    for field in records[0].keys():
        if field == 'id':
            golden_record[field] = records[0][field]
        else:
            values = [record[field] for record in records if pd.notna(record[field])]
            if values:
                golden_record[field] = values[0]  # Keep the first non-null value
    return golden_record

def apply_survivorship_rules(records: List[Dict[str, Any]]) -> Dict[str, Any]:
    golden_record = {}
    for field in records[0].keys():
        if field == 'id':
            golden_record[field] = records[0][field]
        else:
            values = [record[field] for record in records if pd.notna(record[field])]
            if values:
                if field == 'email':
                    golden_record[field] = next((v for v in values if 'example.com' in v), values[0])
                else:
                    golden_record[field] = max(values, key=lambda x: len(str(x)))
    return golden_record

In [10]:
def create_golden_record_and_update_status(df: pd.DataFrame, rule_type: RuleType) -> tuple:
    if df.empty:
        return pd.DataFrame(), pd.DataFrame()

    # Create a single golden record
    all_records = df.to_dict('records')
    
    if rule_type == RuleType.conflict_resolution:
        golden_record = resolve_conflicts(all_records)
    elif rule_type == RuleType.data_aggregation:
        golden_record = aggregate_data(all_records)
    elif rule_type == RuleType.survivorship:
        golden_record = apply_survivorship_rules(all_records)
    else:
        raise ValueError(f"Invalid rule type: {rule_type}")

    golden_record_df = pd.DataFrame([golden_record])

    # Identify contributing records
    contributing_records = df.apply(
        lambda row: any(row[field] == golden_record[field] for field in golden_record.keys() if field != 'id'),
        axis=1
    )

    # Create a copy of the DataFrame and update only the contributing records
    updated_df = df.copy()
    updated_df.loc[contributing_records, 'status'] = 'end-dated'
    updated_df.loc[contributing_records, 'reference_tag'] = 'Referenced'
    updated_df.loc[contributing_records, 'end_date'] = datetime.now()

    return golden_record_df, updated_df

In [11]:
def safe_convert(val):
    """Safely convert values to a string format that avoids encoding issues."""
    try:
        return str(val).encode('utf-8', errors='ignore').decode('utf-8')
    except Exception as e:
        return None

In [12]:
def save_to_mongodb(df: pd.DataFrame, collection_name: str):
    if not df.empty:
        collection = db[collection_name]
        collection.delete_many({})  # Clear existing records

        # Apply safe conversion to avoid encoding issues
        df = df.applymap(safe_convert)
        
        # Convert DataFrame to list of dicts, handling NaT values
        records = json.loads(df.to_json(orient='records', date_format='iso'))
        
        collection.insert_many(records)
        print(f"Data saved to collection: {collection_name}")
    else:
        print(f"No data to save for collection: {collection_name}")

In [13]:
def main():
    # Load and normalize data from all collections
    crm_df = normalize_columns(load_data('crm_system'), 'customer_id', 'name', 'email', phone='phone')
    erp_df = normalize_columns(load_data('erp_system'), 'customer_id', 'name', 'email', billing_address='address')
    ecommerce_df = normalize_columns(load_data('ecommerce_system'), 'user_id', 'name', 'email', shipping_address='address')

    # Combine all data into a single DataFrame
    combined_df = pd.concat([crm_df, erp_df, ecommerce_df], ignore_index=True)

    # Ask for the rule type
    rule_type = input("Enter the rule type (survivorship/conflict_resolution/data_aggregation): ").lower()
    if rule_type not in [r.value for r in RuleType]:
        raise ValueError("Invalid rule type")

    # Apply the selected rule and create a single golden record
    golden_record_df, updated_df = create_golden_record_and_update_status(combined_df, RuleType(rule_type))

    # Save golden record and updated records back to MongoDB
    save_to_mongodb(golden_record_df, 'Golden_Record')
    save_to_mongodb(updated_df, 'Updated_Records')

    print("\nGolden Record:")
    print(golden_record_df)

    print("\nUpdated Records with statuses:")
    print(updated_df)

    print(f"Number of records in Golden Record: {len(golden_record_df)}")
    print(f"Number of records in Updated Records: {len(updated_df)}")
    print(f"Records marked as end-dated: {updated_df['status'].eq('end-dated').sum()}")

    print("Process completed successfully.")

if __name__ == "__main__":
    main()

Data saved to collection: Golden_Record
Data saved to collection: Updated_Records

Golden Record:
                        _id    id        name                 email  \
0  66cedd48983b8bf3f5406b79  C001  John Smith  john.smith@email.com   

          phone             billing_address  credit_limit last_purchase_date  \
0  123-456-7890  123 Main St, City1, State1        5000.0         2023-02-20   

                shipping_address                     address  
0  456 Oak Avenue, City2, State2  123 Main St, City1, State1  

Updated Records with statuses:
                         _id       id            name  \
0   66cedd48983b8bf3f5406b79     C001      John Smith   
1   66cedd48983b8bf3f5406b7a     C002        Jane Doe   
2   66cedd48983b8bf3f5406b7b     C003    Mike Johnson   
3   66cedd48983b8bf3f5406b7c     C004     Emily Brown   
4   66cedd48983b8bf3f5406b7d     C005       David Lee   
5   66cedd48983b8bf3f5406b7e     C006    Sarah Wilson   
6   66cedd48983b8bf3f5406b7f     C007    

C:\Users\Admin\AppData\Local\Temp\ipykernel_5056\2171395739.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(safe_convert)
